In [0]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [0]:
# Sitió web de venta al por menor:
# Regresión logística
ubica_archivo2 = '/FileStore/tables/Log_Reg_dataset.csv'
df = spark.read.csv(ubica_archivo2, inferSchema = True, header = True)
print((df.count(),len(df.columns)))

(20000, 6)

In [0]:
df.printSchema()

root
-- Country: string (nullable = true)
-- Age: integer (nullable = true)
-- Repeat_Visitor: integer (nullable = true)
-- Platform: string (nullable = true)
-- Web_pages_viewed: integer (nullable = true)
-- Status: integer (nullable = true)

In [0]:
df.show(5)
df.columns

+---------+---+--------------+--------+----------------+------+
 Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---------+---+--------------+--------+----------------+------+
 India| 41| 1| Yahoo| 21| 1|
 Brazil| 28| 1| Yahoo| 5| 0|
 Brazil| 40| 0| Google| 3| 0|
Indonesia| 31| 1| Bing| 15| 1|
 Malaysia| 32| 0| Google| 15| 1|
+---------+---+--------------+--------+----------------+------+
only showing top 5 rows

Out[13]: ['Country', 'Age', 'Repeat_Visitor', 'Platform', 'Web_pages_viewed', 'Status']

In [0]:
# Status: Compró el cliente o no
# Age:edad
# Repeat_visitor: visitante repetido o visitante por primera vez
# Plataforma de busqueda usada: Google, Yahoo, Bing
# Número de páginas web en el sitio web
  

In [0]:
#Exploración de datos
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
summary| Country| Age| Repeat_Visitor|Platform| Web_pages_viewed| Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
 count| 20000| 20000| 20000| 20000| 20000| 20000|
 mean| null| 28.53955| 0.5029| null| 9.5533| 0.5|
 stddev| null|7.888912950773227|0.500004090187782| null|6.073903499824976|0.5000125004687693|
 min| Brazil| 17| 0| Bing| 1| 0|
 max|Malaysia| 111| 1| Yahoo| 29| 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+

In [0]:
df.groupBy('Country').count().show()
df.groupBy('Platform').count().show()
df.groupBy('Status').count().show()

+---------+-----+
 Country|count|
+---------+-----+
 Malaysia| 1218|
 India| 4018|
Indonesia|12178|
 Brazil| 2586|
+---------+-----+

+--------+-----+
Platform|count|
+--------+-----+
 Yahoo| 9859|
 Bing| 4360|
 Google| 5781|
+--------+-----+

+------+-----+
Status|count|
+------+-----+
 1|10000|
 0|10000|
+------+-----+

In [0]:
df.groupBy('Status').mean().show()

+------+--------+-------------------+---------------------+-----------+
Status|avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|avg(Status)|
+------+--------+-------------------+---------------------+-----------+
 1| 26.5435| 0.7019| 14.5617| 1.0|
 0| 30.5356| 0.3039| 4.5449| 0.0|
+------+--------+-------------------+---------------------+-----------+

In [0]:
#converting categorical data to numerical form
from pyspark.ml.feature import StringIndexer
search_engine_indexer = StringIndexer(inputCol="Platform", outputCol="Search_Engine_Num").fit(df) # Indexar los sitios de busqueda no suele ser una buena idea
df = search_engine_indexer.transform(df)
df.show(5)

+---------+---+--------------+--------+----------------+------+-----------------+
 Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|
+---------+---+--------------+--------+----------------+------+-----------------+
 India| 41| 1| Yahoo| 21| 1| 0.0|
 Brazil| 28| 1| Yahoo| 5| 0| 0.0|
 Brazil| 40| 0| Google| 3| 0| 1.0|
Indonesia| 31| 1| Bing| 15| 1| 2.0|
 Malaysia| 32| 0| Google| 15| 1| 1.0|
+---------+---+--------------+--------+----------------+------+-----------------+
only showing top 5 rows

In [0]:
df.groupBy('Search_Engine_Num').count().show()

+-----------------+-----+
Search_Engine_Num|count|
+-----------------+-----+
 0.0| 9859|
 1.0| 5781|
 2.0| 4360|
+-----------------+-----+

In [0]:
search_engine_encoder = OneHotEncoder(inputCol="Search_Engine_Num", outputCol="Search_Engine_Vector")
#search_engine_encoder.setDropLast(False)
df = search_engine_encoder.fit(df).transform(df)
df.show(3,False) # El false es paqra que no trunque el contenido de las columnas

+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|Search_Engine_Vector|
+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
India |41 |1 |Yahoo |21 |1 |0.0 |(2,[0],[1.0]) |
Brazil |28 |1 |Yahoo |5 |0 |0.0 |(2,[0],[1.0]) |
Brazil |40 |0 |Google |3 |0 |1.0 |(2,[1],[1.0]) |
+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
only showing top 3 rows

In [0]:
df.groupBy('Platform').count().orderBy('count',ascending=False).show(5,False)
df.groupBy('Search_Engine_Num').count().orderBy('count',ascending=False).show(5,False)

+--------+-----+
Platform|count|
+--------+-----+
Yahoo |9859 |
Google |5781 |
Bing |4360 |
+--------+-----+

+-----------------+-----+
Search_Engine_Num|count|
+-----------------+-----+
0.0 |9859 |
1.0 |5781 |
2.0 |4360 |
+-----------------+-----+

In [0]:
df.groupBy('Platform').count().orderBy('count',ascending=False).show(5,False)
df.groupBy('Search_Engine_Vector').count().orderBy('count',ascending=False).show(5,False)

+--------+-----+
Platform|count|
+--------+-----+
Yahoo |9859 |
Google |5781 |
Bing |4360 |
+--------+-----+

+--------------------+-----+
Search_Engine_Vector|count|
+--------------------+-----+
(2,[0],[1.0]) |9859 |
(2,[1],[1.0]) |5781 |
(2,[],[]) |4360 |
+--------------------+-----+

In [0]:
# Indexer (no recomendado)
df.groupBy('Platform').count().orderBy('count',ascending=False).show(5,False)
df.groupBy('Search_Engine_Num').count().orderBy('count',ascending=False).show(5,False)
df.groupBy('Search_Engine_Vector').count().orderBy('count',ascending=False).show(5,False)
country_indexer = StringIndexer(inputCol="Country", outputCol="Country_Num").fit(df)
country_indexer = StringIndexer(inputCol="Country", outputCol="Country_Num").fit(df)
df = country_indexer.transform(df)
df.select(['Country','Country_Num']).show(3,False)


+--------+-----+
Platform|count|
+--------+-----+
Yahoo |9859 |
Google |5781 |
Bing |4360 |
+--------+-----+

+-----------------+-----+
Search_Engine_Num|count|
+-----------------+-----+
0.0 |9859 |
1.0 |5781 |
2.0 |4360 |
+-----------------+-----+

+--------------------+-----+
Search_Engine_Vector|count|
+--------------------+-----+
(2,[0],[1.0]) |9859 |
(2,[1],[1.0]) |5781 |
(2,[],[]) |4360 |
+--------------------+-----+

+-------+-----------+
Country|Country_Num|
+-------+-----------+
India |1.0 |
Brazil |2.0 |
Brazil |2.0 |
+-------+-----------+
only showing top 3 rows

In [0]:
# one hot encoding
country_encoder = OneHotEncoder(inputCol="Country_Num", outputCol="Country_Vector")
df = country_encoder.fit(df).transform(df)
df.select(['Country','country_Num','Country_Vector']).show(3,False)

+-------+-----------+--------------+
Country|country_Num|Country_Vector|
+-------+-----------+--------------+
India |1.0 |(3,[1],[1.0]) |
Brazil |2.0 |(3,[2],[1.0]) |
Brazil |2.0 |(3,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows

In [0]:
df.groupBy('Country').count().orderBy('count',ascending=False).show(5,False)
#df.groupBy('Country_Num').count().orderBy('count',ascending=False).show(5,False)
df.groupBy('Country_Vector').count().orderBy('count',ascending=False).show(5,False)


+---------+-----+
Country |count|
+---------+-----+
Indonesia|12178|
India |4018 |
Brazil |2586 |
Malaysia |1218 |
+---------+-----+

+--------------+-----+
Country_Vector|count|
+--------------+-----+
(3,[0],[1.0]) |12178|
(3,[1],[1.0]) |4018 |
(3,[2],[1.0]) |2586 |
(3,[],[]) |1218 |
+--------------+-----+

In [0]:
# Integrar la información para correr el modelo
df_assembler = VectorAssembler(inputCols=['Search_Engine_Vector','Country_Vector','Age', 'Repeat_Visitor','Web_pages_viewed'], outputCol="features")
df = df_assembler.transform(df)
df.printSchema()

root
-- Country: string (nullable = true)
-- Age: integer (nullable = true)
-- Repeat_Visitor: integer (nullable = true)
-- Platform: string (nullable = true)
-- Web_pages_viewed: integer (nullable = true)
-- Status: integer (nullable = true)
-- Search_Engine_Num: double (nullable = false)
-- Search_Engine_Vector: vector (nullable = true)
-- Country_Num: double (nullable = false)
-- Country_Vector: vector (nullable = true)
-- features: vector (nullable = true)

In [0]:
df.show(n = 5)

+---------+---+--------------+--------+----------------+------+-----------------+--------------------+-----------+--------------+--------------------+
 Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|Search_Engine_Vector|Country_Num|Country_Vector| features|
+---------+---+--------------+--------+----------------+------+-----------------+--------------------+-----------+--------------+--------------------+
 India| 41| 1| Yahoo| 21| 1| 0.0| (2,[0],[1.0])| 1.0| (3,[1],[1.0])|[1.0,0.0,0.0,1.0,...|
 Brazil| 28| 1| Yahoo| 5| 0| 0.0| (2,[0],[1.0])| 2.0| (3,[2],[1.0])|[1.0,0.0,0.0,0.0,...|
 Brazil| 40| 0| Google| 3| 0| 1.0| (2,[1],[1.0])| 2.0| (3,[2],[1.0])|(8,[1,4,5,7],[1.0...|
Indonesia| 31| 1| Bing| 15| 1| 2.0| (2,[],[])| 0.0| (3,[0],[1.0])|(8,[2,5,6,7],[1.0...|
 Malaysia| 32| 0| Google| 15| 1| 1.0| (2,[1],[1.0])| 3.0| (3,[],[])|(8,[1,5,7],[1.0,3...|
+---------+---+--------------+--------+----------------+------+-----------------+--------------------+-----------+--------------+--------------------+
only showing top 5 rows

In [0]:
df.select(['features','Status']).show(10,False)

+-----------------------------------+------+
features |Status|
+-----------------------------------+------+
[1.0,0.0,0.0,1.0,0.0,41.0,1.0,21.0]|1 |
[1.0,0.0,0.0,0.0,1.0,28.0,1.0,5.0] |0 |
(8,[1,4,5,7],[1.0,1.0,40.0,3.0]) |0 |
(8,[2,5,6,7],[1.0,31.0,1.0,15.0]) |1 |
(8,[1,5,7],[1.0,32.0,15.0]) |1 |
(8,[1,4,5,7],[1.0,1.0,32.0,3.0]) |0 |
(8,[1,4,5,7],[1.0,1.0,32.0,6.0]) |0 |
(8,[1,2,5,7],[1.0,1.0,27.0,9.0]) |0 |
(8,[0,2,5,7],[1.0,1.0,32.0,2.0]) |0 |
(8,[2,5,6,7],[1.0,31.0,1.0,16.0]) |1 |
+-----------------------------------+------+
only showing top 10 rows

In [0]:
model_df=df.select(['features','Status'])
model_df.show(n = 5)

+--------------------+------+
 features|Status|
+--------------------+------+
[1.0,0.0,0.0,1.0,...| 1|
[1.0,0.0,0.0,0.0,...| 0|
(8,[1,4,5,7],[1.0...| 0|
(8,[2,5,6,7],[1.0...| 1|
(8,[1,5,7],[1.0,3...| 1|
+--------------------+------+
only showing top 5 rows

In [0]:
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [0]:
print(training_df.count())
training_df.groupBy('Status').count().show()

14938
+------+-----+
Status|count|
+------+-----+
 1| 7538|
 0| 7400|
+------+-----+

In [0]:
print(test_df.count())
test_df.groupBy('Status').count().show()

5062
+------+-----+
Status|count|
+------+-----+
 1| 2462|
 0| 2600|
+------+-----+

In [0]:
log_reg=LogisticRegression(labelCol='Status').fit(training_df)

In [0]:
# No muy útiles
train_results=log_reg.evaluate(training_df).predictions
train_results.show(n=5)

+--------------------+------+--------------------+--------------------+----------+
 features|Status| rawPrediction| probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
(8,[0,2,5,7],[1.0...| 0|[5.91956295446054...|[0.99732082328244...| 0.0|
(8,[0,2,5,7],[1.0...| 0|[5.91956295446054...|[0.99732082328244...| 0.0|
(8,[0,2,5,7],[1.0...| 0|[5.91956295446054...|[0.99732082328244...| 0.0|
(8,[0,2,5,7],[1.0...| 0|[5.91956295446054...|[0.99732082328244...| 0.0|
(8,[0,2,5,7],[1.0...| 0|[5.16818638716438...|[0.99433736031687...| 0.0|
+--------------------+------+--------------------+--------------------+----------+
only showing top 5 rows

In [0]:
results=log_reg.evaluate(test_df).predictions
results.printSchema() 

root
-- features: vector (nullable = true)
-- Status: integer (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
results.select(['Status','prediction']).show(10,False)

+------+----------+
Status|prediction|
+------+----------+
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
0 |0.0 |
+------+----------+
only showing top 10 rows

In [0]:
tp = results[(results.Status == 1) & (results.prediction== 1)].count()
tn = results[(results.Status == 0) & (results.prediction == 0)].count()
fp = results[(results.Status == 0) & (results.prediction== 1)].count()
fn = results[(results.Status == 1) & (results.prediction == 0)].count()

In [0]:
fn

Out[53]: 144

In [0]:
accuracy=float((tp+tn) /(results.count()))
print(accuracy)

0.9425128407743975

In [0]:
# sensibilidad o recall
recall = float(tp)/(tp + fn)
print(recall)

0.9415109666937449

In [0]:
# Precision: número de los valores predicihos positivamente que en verdad son observaciones positivas (TP / (TP + FP))
precision = float(tp) / (tp + fn)
print(precision)

0.9415109666937449